In [5]:
import os, sys, csv
from datetime import date
import glob

In [6]:
BASE_DIR = '/'
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "documenter.settings.setup")

'documenter.settings.setup'

In [7]:
import django
import lorem
django.setup()

from rate_doc.models import * 
from django.contrib.auth.models import User

In [9]:
User.objects.all()

<QuerySet [<User: admin>, <User: bjgoode>, <User: aida1>, <User: chadlevinson>]>

In [5]:
articles = glob.glob('../docs/NYTimes/articles/*.txt')

for fn in articles:
    with open(fn) as file:
        text = file.read()
        
    text = text.strip()
    text_split = text.split('\n')
    title = text_split[0]
    
    author = text_split[1]
    if 'By' in author:
        author = author.replace(' With ', ' and ')
        author = author.split('By')[1].strip().split(' and ')
    if ',' in author or author == '':
        author = ['No Author Given']
        
    date = text_split[2]

    report_text = '\n'.join(text_split[3:])
    
    author_obj = [Author.objects.get_or_create(name=a)[0] for a in author]
    mediaType_obj, created = MediaType.objects.get_or_create(type_desc='Newspaper')
    media_obj, created = MediaOrg.objects.get_or_create(media_type=mediaType_obj, org_name='New York Times')

    report, created = Report.objects.get_or_create(
        id = int(fn.split('/')[-1].split('-')[0]),
        title=title,
        media_org = media_obj,
        report_text_html = report_text
    )
    report.author.set(author_obj)
    report.save()

In [12]:
#users = User.objects.all()
users = User.objects.filter(username = 'chadlevinson')
print(users)

<QuerySet [<User: chadlevinson>]>


In [13]:
for usr in users:
    for r in Report.objects.all():
        Review.objects.get_or_create(
            assignedTo = usr,
            report = r,
            review_html = r.report_text_html,
        )